## Group the dataframe based on the hotel_id and lp_id, ( both of it) and Merge it and Export at the END

# hotel_id

In [1]:
import polars as pl
import datetime as dt

import os
import sys

new_path = "/".join(os.getcwd().split("/")[:-1])
sys.path.append(new_path)


rooms_csv = pl.read_csv(f"{new_path}/data/updated_core_rooms.csv", try_parse_dates=True)
reference_csv = pl.read_csv(f"{new_path}/data/referance_rooms-1737378184366.csv", try_parse_dates=True)
print(f"rooms_csv : {rooms_csv.columns}")
print(f"reference_csv : {reference_csv.columns}")

rooms_csv : ['core_room_id', 'core_hotel_id', 'lp_id', 'supplier_room_id', 'supplier_name', 'supplier_room_name']
reference_csv : ['hotel_id', 'lp_id', 'room_id', 'room_name']


In [29]:

reference_csv_struct = reference_csv.with_columns([
    pl.struct(["lp_id", "room_id", "room_name"]).alias("data")
])

reference_csv_grouped = reference_csv_struct.group_by("hotel_id").agg(pl.col("data"))

reference_csv_grouped.head()

hotel_id,data
i64,list[struct[3]]
10161342,"[{""lp918f6"",1016134202,""Holiday Home""}]"
13596955,"[{""lp1b8b94"",1142577252,""Economy Cabin""}, {""lp1b8b94"",1142577260,""Family Cabin""}]"
10097744,"[{""lp655a5479"",1009774402,""Holiday Home""}]"
1383045,"[{""lp6569f874"",138304501,""Apartment with Garden View""}]"
9775581,"[{""lp6567831f"",977558101,""Budget Double Room""}, {""lp6567831f"",977558102,""Budget Double Room""}, {""lp6567831f"",977558103,""Budget Double Room""}]"


In [31]:

rooms_csv_struct = rooms_csv.with_columns([
    pl.struct(['core_room_id', 'lp_id', 'supplier_room_id', 'supplier_room_name']).alias("data")
])

rooms_csv_grouped = rooms_csv_struct.group_by("core_hotel_id").agg(pl.col("data"))

rooms_csv_grouped.head()

core_hotel_id,data
i64,list[struct[4]]
1701604558,"[{2570391,""lp656c6cce"",322644591,""House, 5 Bedrooms""}]"
632858,"[{45781,""lp9a81a"",201711726,""Condo, 2 Bedrooms""}]"
1700228477,"[{1220173,""lp65576d7d"",315661656,""Basic Room""}]"
1700299006,"[{1421968,""lp655880fe"",320936204,""Apartment (Volpe)""}]"
632471,"[{1422552,""lp9a697"",201711648,""Apartment, 4 Bedrooms""}]"


In [32]:
len(rooms_csv_grouped), len(reference_csv_grouped)

(813834, 40011)

In [33]:
merged_df = reference_csv_grouped.join(rooms_csv_grouped, left_on="hotel_id", right_on="core_hotel_id", how="inner")
merged_df



hotel_id,data,data_right
i64,list[struct[3]],list[struct[4]]
493688,"[{""lp655fc660"",49368801,""Super Deluxe Double Room""}, {""lp655fc660"",49368803,""Deluxe Double Room""}, … {""lp655fc660"",49368805,""Quadruple Room with Fan and Private External Bathroom""}]","[{2840042,""lp78878"",314319324,""Penthouse""}, {2840050,""lp78878"",314319269,""Solo room""}, … {2841415,""lp78878"",314319275,""Classic room""}]"
1022558,"[{""lpe0156"",102255801,""Two-Bedroom Holiday Home""}]","[{825765,""lpf9a5e"",218128589,""Deluxe Double or Twin Room, 1 King Bed""}]"
358635,"[{""lp655a7d4a"",1143148244,""Superior Double Room with Balcony""}, {""lp655a7d4a"",1143148245,""Large Double Room""}, … {""lp655a7d4a"",1143148251,""Budget Double Room""}]","[{399836,""lp578eb"",200295638,""Apartment 3 Bedrooms Suite""}, {407269,""lp578eb"",200760470,""Apartment 2 Bedrooms Suite""}, … {2679523,""lp578eb"",200248359,""Executive Suite Room""}]"
343311,"[{""lp6e412"",34331101,""Four-Bedroom Villa with Private Pool""}, {""lp6e412"",34331102,""One-Bedroom Villa""}, … {""lp6e412"",34331107,""Two-Bedroom Villa""}]","[{1068076,""lp53d0f"",215448941,""Japanese Room 6 tatami City View""}, {1076408,""lp53d0f"",215448943,""Japanese Room 8 tatami River View""}]"
1678380,"[{""lp655ab43c"",167838001,""Twin Room with Garden View""}, {""lp655ab43c"",167838002,""Deluxe Double Room with Shower""}]","[{899999,""lp199c2c"",218431622,""Studio, Non Smoking""}]"
…,…,…
502262,"[{""lp655a174f"",50226201,""Double or Twin Room""}]","[{932273,""lp7a9f6"",200950739,""Twin Room""}, {938925,""lp7a9f6"",200950736,""Double Room""}, {945236,""lp7a9f6"",200950737,""Family Room (Large)""}]"
652481,"[{""lp65669a08"",65248104,""Two-Bedroom Apartment""}]","[{1570960,""lp9f4c1"",201790985,""Double Room, Balcony, Lake View""}, {1576587,""lp9f4c1"",201790995,""Romantic Bungalow, 1 Bedroom""}, … {1586592,""lp9f4c1"",201790997,""Luxury Bungalow, 1 Bedroom, Garden Area""}]"
1044441,"[{""lp65679d74"",104444114,""Deluxe Room""}, {""lp65679d74"",104444115,""Superior Room""}, … {""lp65679d74"",104444117,""Presidential Suite""}]","[{1280030,""lpfefd9"",215420594,""Tent (Sultana)""}, {1284911,""lpfefd9"",215420595,""Deluxe Tent (Titrite)""}, … {1295656,""lpfefd9"",215420588,""Tent (Itri)""}]"


In [40]:
for row in merged_df.filter(pl.col("hotel_id") == 358635).iter_rows(named=True):
    print("reference")
    for i in row["data"]:
        print(i)
    print("rooms")
    for i in row["data_right"]:
        print(i)
    

reference
{'lp_id': 'lp655a7d4a', 'room_id': 1143148244, 'room_name': 'Superior Double Room with Balcony'}
{'lp_id': 'lp655a7d4a', 'room_id': 1143148245, 'room_name': 'Large Double Room'}
{'lp_id': 'lp655a7d4a', 'room_id': 1143148246, 'room_name': 'Standard Double Room with Balcony'}
{'lp_id': 'lp655a7d4a', 'room_id': 1143148247, 'room_name': 'Comfort Single Room'}
{'lp_id': 'lp655a7d4a', 'room_id': 1143148248, 'room_name': 'Deluxe Double Room with Balcony'}
{'lp_id': 'lp655a7d4a', 'room_id': 1143148249, 'room_name': 'Family Two-Bedroom Suite'}
{'lp_id': 'lp655a7d4a', 'room_id': 1143148250, 'room_name': 'Superior Deluxe Double Room '}
{'lp_id': 'lp655a7d4a', 'room_id': 1143148251, 'room_name': 'Budget Double Room'}
rooms
{'core_room_id': 399836, 'lp_id': 'lp578eb', 'supplier_room_id': 200295638, 'supplier_room_name': 'Apartment 3 Bedrooms Suite'}
{'core_room_id': 407269, 'lp_id': 'lp578eb', 'supplier_room_id': 200760470, 'supplier_room_name': 'Apartment 2 Bedrooms Suite'}
{'core_room_i

# lp_id

In [2]:

reference_csv_struct2 = reference_csv.with_columns([
    pl.struct(["hotel_id", "room_id", "room_name"]).alias("data")
])

reference_csv_grouped2 = reference_csv_struct2.group_by("lp_id").agg(pl.col("data"))

reference_csv_grouped2.head()

lp_id,data
str,list[struct[3]]
"""lp993e9""","[{13947201,1143640765,""Standard Room""}, {13947201,1143640808,""Executive Penthouse""}, … {13947201,1143640906,""Executive Room""}]"
"""lpc355a""","[{13815241,1143383965,""Condo, Multiple Beds, Lake View (Lakeside 1495)""}]"
"""lp96db5""","[{9637251,963725102,""Holiday Home""}]"
"""lp656c5854""","[{13936281,1143592316,""House, 3 Bedrooms""}]"
"""lp32223""","[{1337173,133717301,""One-Bedroom Apartment with Balcony""}]"


In [3]:

rooms_csv_struct2 = rooms_csv.with_columns([
    pl.struct(['core_room_id', 'core_hotel_id', 'supplier_room_id', 'supplier_room_name']).alias("data")
])

rooms_csv_grouped2 = rooms_csv_struct2.group_by("lp_id").agg(pl.col("data"))

rooms_csv_grouped2.head()

lp_id,data
str,list[struct[4]]
"""lp655710b1""","[{1453747,1700204721,314109329,""Apartment""}]"
"""lp365212""","[{665895,3559954,213635407,""Standard Twin Room""}, {671749,3559954,213638787,""Superior Double Room""}]"
"""lp43c54""","[{363542,277588,498859,""Grand Room""}, {371549,277588,200276819,""Junior Suite""}, … {386781,277588,394605,""Deluxe King Room""}]"
"""lp6555685d""","[{356534,1700096093,217285171,""Apartment, 2 Bedrooms""}]"
"""lp65573e04""","[{694082,1700216324,314450992,""Apartment""}]"


In [5]:
merged_df2 = reference_csv_grouped2.join(rooms_csv_grouped2, on="lp_id", how="inner")
merged_df2.shape


(28638, 3)

In [6]:
merged_df2 = reference_csv_grouped2.join(rooms_csv_grouped2, on="lp_id", how="left")
merged_df2.shape

(40011, 3)

In [79]:
count = 100 
check_lp_ids = []
for row in merged_df2.iter_rows(named=True):
    if len(row["data"]) > 3:
        check_lp_ids.append(row["lp_id"])
        count -=1
    if not count:
        break

In [80]:

def visualize(lp_id):
    for row in merged_df2.filter(pl.col("lp_id") == lp_id).iter_rows(named=True):
        print("referenceCatalog: (Standardized catalog): ", lp_id)
        for i in sorted(row["data"], key=lambda x: x['room_name']):
            print(i)
        print("inputCatalog: (Supplier's provided catalog)", lp_id)
        for i in sorted(row["data_right"], key=lambda x: x['supplier_room_name']):
            print(i)

for lp_id in check_lp_ids[50:60]:
    visualize(lp_id)
    print("--------------------------"*6)

referenceCatalog: (Standardized catalog):  lp65577d96
{'hotel_id': 13619378, 'room_id': 1142331723, 'room_name': 'Basic Family Room, Non Smoking'}
{'hotel_id': 13619378, 'room_id': 1142331665, 'room_name': 'Deluxe Japanese Room, Non Smoking (Basement floors may be assigned)'}
{'hotel_id': 13619378, 'room_id': 1142331653, 'room_name': 'Grand Japanese Room, Non Smoking (Basement floors may be assigned)'}
{'hotel_id': 13619378, 'room_id': 1142331605, 'room_name': 'Japanese Concept Room, Non Smoking'}
{'hotel_id': 13619378, 'room_id': 1142331703, 'room_name': 'Run of House, Non smoking (Bed Type not guaranteed)'}
{'hotel_id': 13619378, 'room_id': 1142331714, 'room_name': 'Standard Family Room - Kawayuka Dining with Sukiyaki Dinner Non Smoking'}
{'hotel_id': 13619378, 'room_id': 1142331549, 'room_name': 'Standard Family Room, Non Smoking'}
{'hotel_id': 13619378, 'room_id': 1142331690, 'room_name': 'Standard Family room +Tea Ceremony Experience with Maiko Starts at 6pm,Non Smoking'}
{'hotel_

# EXPORT as json

In [82]:
data = []
for row in merged_df2.iter_rows(named=True):
    reference = []
    supplier = []
    for r in row["data"]:
        reference.append(r["room_name"])
    for s in row["data_right"]:
        supplier.append(s["supplier_room_name"])
    data.append({"reference": reference, "supplier": supplier})

In [87]:
import json
with open("../data/merged.json", "w") as f:
    json.dump(data , f)

In [88]:
with open("../data/merged.json", "r") as f:
    data_r = json.load(f)

In [ ]:
data_r